In [52]:
import pandas as pd
from datetime import datetime
import json
import random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import time

with open('pass.json') as file:
    user = json.load(file)
    
MAIN_URL = 'https://www.linkedin.com'
CIUDADES = ['bogota', 'medellin', 'cali', 'barranquilla', 'bucaramanga']
CIUDADES = ['bogota', 'medellin']
CRITERIOS = ['busqueda', 'busco', 'busca', 'buscan', 'buscando', 'buscamos']
CRITERIOS = ['busqueda', 'busco']
XPATH_URL_BOX = './/div[@class="entity-result__insights t-12"]/a'
XPATH_NAME = './/div[@class="linked-area cursor-pointer"]//a[@class="app-aware-link"]'
XPATH_SUBIDO = './/div[@class="linked-area cursor-pointer"]//p'
XPATH_TEXT = './/div[@class="entity-result__content-inner-container"]//p'
XPATH_BOXES = '//div[@class="entity-result"]'
XPATH_KEY = 'session_key'
XPATH_PASS ='session_password'
XPATH_BUTTON = 'sign-in-form__submit-button'
DIAS_MAX = 2
PAGES = 10


def links_para_buscar():
    links = {}
    for ciudad in CIUDADES:
        links[ciudad] = {}
        for criterio in CRITERIOS:
            links[ciudad][criterio] = []
            for page in range(PAGES):
                links[ciudad][criterio].append(f'https://www.linkedin.com/search/results/content/?keywords={ciudad}%20%20{criterio}&origin=FACETED_SEARCH&page={page + 1}&sortBy=%22date_posted%22')
    return links

def open_sesion():
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')
    options.add_argument('--no-sandbox')

    time.sleep(1)
    driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
    driver.get(MAIN_URL)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH , '//button[@class="sign-in-form__submit-button"]')))

    user_name = driver.find_element_by_id(XPATH_KEY)
    user_name.send_keys(user['name'])
    password = driver.find_element_by_id(XPATH_PASS)
    password.send_keys(user['pass'])

    login_button = driver.find_element_by_class_name(XPATH_BUTTON)
    login_button.click()
    time.sleep(random.uniform(10, 15))
    return driver

def validate_time(tiempo):
    flag = False
    if tiempo.find('días') != -1 and int(tiempo.split()[0]) > DIAS_MAX:
        flag = True
    elif tiempo.find('mes') != -1 or tiempo.find('año') != -1 or tiempo.find('meses') != -1 or tiempo.find('semana') != -1:
        flag = True
    return flag

def extraer_data(boxes, data, ciudad):
    for box in boxes:
        try:
            url = box.find_element_by_xpath(XPATH_URL_BOX).get_attribute('href')
            name = box.find_element_by_xpath(XPATH_NAME).text
            subido = box.find_element_by_xpath(XPATH_SUBIDO).text.replace('•','').strip()
            text_box = box.find_element_by_xpath(XPATH_TEXT).text
            if validate_time(subido):
                return data, True
            
            data = data.append({'Organizacion': name, 'Texto': text_box, 'Link': url, 'Tiempo de publicado': subido, 
                                 'Ciudad': ciudad}, ignore_index=True)
        
        except:
            print(f'Un registro no guardadon, URL: {url}')
    return data, False

def open_links(driver, links, data):
    total_links = len(links) * len(links['bogota']) * len(links['bogota']['busqueda'])
    avance = 0
    for ciudad in links:
        for criterio in links[ciudad]:
            for url in links[ciudad][criterio]:
                avance += 1
                print(f'Links {avance} de {total_links}')
                try:
                    driver.get(url)
                    time.sleep(random.uniform(2, 5))
                    boxes = driver.find_elements_by_xpath(XPATH_BOXES)
                    data, flag = extraer_data(boxes, data, ciudad)
                    if flag:
                        break
                except:
                    print(f'URL: {url} no cargo') 
    return data

def save(data):
    date_name = f'{datetime.today().day}-{datetime.today().month}-{datetime.today().year}'
    data.to_excel(f'Linkedin_{date_name}.xlsx', index=False)

def run(data):
    driver = open_sesion()
    links = links_para_buscar()
    data = open_links(driver, links, data)
    save(data)
    driver.close()
    return data
    
if __name__ == '__main__':
    data = pd.DataFrame(columns=['Organizacion', 'Texto', 'Link', 'Tiempo de publicado', 'Ciudad'])
    data = run(data)
    print(data)

<ipython-input-52-de8599a2a80c>:50: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


Links 1 de 40
Un registro no guardadon, URL: https://www.linkedin.com/feed/update/urn:li:activity:6780290088027901952?updateEntityUrn=urn%3Ali%3Afs_feedUpdate%3A%28V2%2Curn%3Ali%3Aactivity%3A6780290088027901952%29
Un registro no guardadon, URL: https://www.linkedin.com/feed/update/urn:li:activity:6780290088027901952?updateEntityUrn=urn%3Ali%3Afs_feedUpdate%3A%28V2%2Curn%3Ali%3Aactivity%3A6780290088027901952%29
Un registro no guardadon, URL: https://www.linkedin.com/feed/update/urn:li:activity:6780282470123266048?updateEntityUrn=urn%3Ali%3Afs_feedUpdate%3A%28V2%2Curn%3Ali%3Aactivity%3A6780282470123266048%29
Un registro no guardadon, URL: https://www.linkedin.com/feed/update/urn:li:activity:6780282470123266048?updateEntityUrn=urn%3Ali%3Afs_feedUpdate%3A%28V2%2Curn%3Ali%3Aactivity%3A6780282470123266048%29
Un registro no guardadon, URL: https://www.linkedin.com/feed/update/urn:li:activity:6780282470123266048?updateEntityUrn=urn%3Ali%3Afs_feedUpdate%3A%28V2%2Curn%3Ali%3Aactivity%3A678028247

In [2]:
import pandas as pd

In [53]:
df = pd.read_excel('Linkedin_18-3-2021.xlsx')
df = df.append(pd.read_excel('Linkedin_19-3-2021.xlsx'))
df = df.append(pd.read_excel('Linkedin_23-3-2021.xlsx'))
df = df.append(pd.read_excel('Linkedin_15-3-2021.xlsx'))

In [55]:
df.sample(4)

Organizacion  \
154                                     LG Consultores   
378  Jorge Berrio Gómez\nVer el perfil de Jorge Ber...   
481  Liliana Andrea Guzman Novoa\nVer el perfil de ...   
388  Wendy Paola Altamar Perez\nVer el perfil de We...   

                                                 Texto  \
154  ¿En busca de trabajo? Esta oportunidad puede s...   
378  Se busca Diseñador(a) de Interiores en la ciud...   
481  ¿Buscan un nivel de vida más alto? No solo hab...   
388  ...empleo porque la mayoría de las empresas o ...   

                                                  Link Tiempo de publicado  \
154  https://www.linkedin.com/feed/update/urn:li:ac...             2 horas   
378  https://www.linkedin.com/feed/update/urn:li:ac...              4 días   
481  https://www.linkedin.com/feed/update/urn:li:ac...             2 meses   
388  https://www.linkedin.com/feed/update/urn:li:ac...               1 mes   

           Ciudad  
154      medellin  
378  barranquilla  
481   bucaramanga  
388  barranquilla

In [57]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def dic_stop():
    stop = stopwords.words('spanish')
    stop.extend(CIUDADES)
    stop.extend(CRITERIOS)
    stop.extend(['bogotá', 'búsqueda', 'siguientes', 'áreas'])
    return stop
    
def preprocesor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    return text

def tokenizer(text):
    return text.split()

def preprocesor_text(text):
    text = preprocesor(text)
    words = [w for w in tokenizer(text) if w not in stop]
    return words

stop = dic_stop()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaort\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [58]:
import numpy as np
list_words = []
for words in list(df['Texto'].apply(preprocesor_text)):
    list_words.extend(words)

unicos, reps = np.unique((list_words), return_counts=True) 

In [89]:
max_reps = []
max_unicos =[]
sort_index = np.argsort(reps)

for i in range(len(sort_index) - 1, -1, -1):
    max_reps.append(reps[sort_index[i]])
    max_unicos.append(unicos[sort_index[i]])

In [115]:
max_unicos

['experiencia',
 'ver',
 'https',
 'barranquilla',
 'in',
 'lnkd',
 'cali',
 'buscamos',
 'medellín',
 'buscando',
 'profesional',
 'bucaramanga',
 'red',
 'ciudad',
 'años',
 'colombia',
 'si',
 'busca',
 'comercial',
 'hashtag',
 'vida',
 'trabajosihay',
 '3',
 'encontramos',
 'empresas',
 'empleo',
 'hola',
 'director',
 'talento',
 'coordinador',
 'trabajo',
 'sector',
 'estimada',
 'perfil',
 'analista',
 '2',
 'oportunidad',
 'ingeniería',
 'buscan',
 'ingeniero',
 'com',
 'empresa',
 'encuentro',
 'industrial',
 'equipo',
 'hoja',
 'proyectos',
 'laboral',
 'mejor',
 'área',
 '1',
 'afines',
 'mínimo',
 'parte',
 'administración',
 'ventas',
 'desarrollo',
 'ser',
 'personas',
 'jefe',
 'compañía',
 'bilingüe',
 '5',
 '000',
 'correo',
 'información',
 'mercadeo',
 'carreras',
 'hoy',
 'cargo',
 'manejo',
 'día',
 'administrativo',
 'gerente',
 'ofertas',
 'gestión',
 's',
 'siguiente',
 'vacante',
 'importante',
 'salario',
 'co',
 'alguien',
 'servicios',
 'financiero',
 'pers

In [1]:
CIUDADES = ['bogota', 'medellin', 'cali', 'barranquilla', 'bucaramanga']
#CIUDADES = ['bogota', 'medellin']
CRITERIOS = ['busqueda', 'busco', 'busca', 'buscan', 'buscando', 'buscamos']

In [ ]:
%config Completer.use_jedi = False